# Create JasonLine files from CSV ground truth


This script will create two labeled datasets: <dataset_name>_training and <dataset_name>_test

1. split training and test labeled datapoints
2. create jason-line files for training and test
3. upload jason-line files to default workspace blobstore, under fhl/datasets/<dataset_name>/label folder
4. register labeled datasets

In [ ]:
import json
import pandas as pd
import sklearn
from sklearn.model_selection import train_test_split


In [ ]:
datastore_name = 'workspaceblobstore'
dataset_name = 'NoWindow'

In [ ]:
# read ground truth from csv file
raw_df = pd.read_csv('./labels/stage_2_detailed_class_info.csv')
dedup_df = raw_df[ (raw_df['patientId'].duplicated(keep='first') == False)]
train, test = train_test_split(dedup_df, test_size = 0.2, random_state = 2021)

In [ ]:
# Create Multi-Class JasonLine Objects
def create_jasonline_objects(datastore_name: str, dataset_name: str, df: pd.DataFrame):
    jsonline_obj = []
    for _, row in df.iterrows():
        obj = {}
        obj['image_url'] = 'AmlDatastore://' + datastore_name + '/fhl/datasets/' + dataset_name + '/image/' + row['patientId'] + '.png'
        obj['label'] = row['class']
        obj['label_confidence'] = 1.0
        jsonline_obj.append(obj)
    return jsonline_obj

In [ ]:
import os
import json
def save_jasonline_file(objects: list, file_name: str):
    base_path = os.path.dirname(file_name)
    if not os.path.exists(base_path):
        os.makedirs(base_path)
    with open(file_name, 'w') as jf:
        for obj in objects:
            jf.write('{}\n'.format(json.dumps(obj)))

In [ ]:
# Generate jsonline files, which can be registered as labeled dataset
training_label_folder = os.path.abspath(os.path.join(os.path.curdir, 'training_label'))
test_label_folder = os.path.abspath(os.path.join(os.path.curdir, 'test_label'))

train_obj = create_jasonline_objects(datastore_name, dataset_name, train)
save_jasonline_file(train_obj, os.path.join(training_label_folder, 'labeleddatapoints_training.jsonl'))

test_obj = create_jasonline_objects(datastore_name, dataset_name, test)
save_jasonline_file(test_obj, os.path.join(test_label_folder, 'labeleddatapoints_test.jsonl'))

In [ ]:
import azureml.core
from azureml.core import Workspace
# Load workspace
ws = Workspace.from_config()
ds = ws.datastores['workspaceblobstore']
ds.upload(src_dir = training_label_folder, target_path= '/fhl/datasets/' + dataset_name + '/label/', overwrite= True)
ds.upload(src_dir = './test_label', target_path= '/fhl/datasets/' + dataset_name + '/label/', overwrite= True)


In [ ]:
# Create/register labeled dataset for training and test(inference)
from azureml.contrib.dataset.labeled_dataset import _LabeledDatasetFactory, LabeledDatasetTask

tags = {}
tags['labelingCreatedBy'] = "Guess"
tags['labelingProjectType'] = 'Image Classification Multi-class'
tags['SourceDatastoreName'] = 'workspaceblobstore'
tags['SourceRelativePath'] = 'fhl/datasets/' + dataset_name + '/image/'
tags['labelingLabelName'] = '["Lung Opacity","No Lung Opacity / Not Normal","Normal"]'

training_dataset = _LabeledDatasetFactory.from_json_lines(task=LabeledDatasetTask.IMAGE_CLASSIFICATION, path=ds.path('fhl/datasets/' + dataset_name + '/label/labeleddatapoints_training.jsonl'))
training_dataset.register(ws, dataset_name + '_training', tags= tags)

test_dataset = _LabeledDatasetFactory.from_json_lines(task=LabeledDatasetTask.IMAGE_CLASSIFICATION, path=ds.path('fhl/datasets/' + dataset_name + '/label/labeleddatapoints_test.jsonl'))
test_dataset.register(ws, dataset_name + '_test', tags= tags)